In [1]:
import os


In [2]:
%pwd

'd:\\Project\\Kidney-Diseas-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Project\\Kidney-Diseas-Classification'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # now this becomes entity
class DataIngestionConfig:
    """Data Ingestion Configuration."""
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [21]:
# updating congiguration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [22]:
import os 
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self)-> str:
        """
        Download the data from the source URL to the local data file.
        """

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from: {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")

        except Exception as e:
             raise e
        

    def extract_zip_file(self):
        """
        Extract the zip file to the unzip directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [24]:
# update pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-07-06 15:59:14,051: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-06 15:59:14,055: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-06 15:59:14,057: INFO: common: created directory at: artifacts]
[2025-07-06 15:59:14,058: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-06 15:59:14,060: INFO: 1462462793: Downloading file from: https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=ff4a4618-6ab0-46eb-b008-abee8ad351a8
To: d:\Project\Kidney-Diseas-Classification\artifacts\data_ingestion\data.zip








































100%|██████████| 57.7M/57.7M [00:06<00:00, 9.52MB/s]

[2025-07-06 15:59:23,228: INFO: 1462462793: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]
